### Introduction
We create a time series data at yearly resolution for each PRIO GRID cell with the development stage of SNL mining on that PRIO GRID cell.

1. Each PRIO GRID 44 rows information with 1 development stage information in each year. Here are the year range: [1970, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
              
       The SNL company doesn't provide that data between 1970-1980

2. If one cell has multiple development stage(which means multiple SNL minings) in the same year, keep the latest stage. 

3. If there is no available data for that year, we use the latest development stage from previous year.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

### Read Prio Grid cell data

In [2]:
spine = pd.read_csv("../input/PRIO GRID spine.csv")
print(spine.shape)
spine.head()

(64818, 3)


gid    lon    lat
0  49182 -69.25 -55.75
1  49183 -68.75 -55.75
2  49184 -68.25 -55.75
3  49185 -67.75 -55.75
4  49186 -67.25 -55.75

### Read SNL data with SNL property ID and PRIO grid ID

In [3]:
df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")
print(df_SNL.shape)
df_SNL.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df_SNL = df_SNL[pd.to_numeric(df_SNL['PRIO GRID ID'], errors='coerce').notnull()]
df_SNL["PRIO GRID ID"] = df_SNL["PRIO GRID ID"].astype("int")
df_SNL = df_SNL[(df_SNL['LATITUDE']<=90) & (df_SNL['LATITUDE']>= -90) &
                (df_SNL['LONGITUDE']<=180) & (df_SNL['LONGITUDE']>= -180)].reset_index().drop(columns="index",axis=1)
print(df_SNL.shape)
df_SNL.head()

(39612, 46)
(39370, 46)


/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18806/2275177157.py:1: DtypeWarning: Columns (2,22,23,24,25,26,27,28,30,31,32,33,36,37,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")


Unnamed: 0           Property name Property ID Also Known As  \
0          0           11 RAU-Novaya       68289           NaN   
1          1           12 Mile Creek       66178           NaN   
2          2               14 Karats       65525           NaN   
3          3  14 Mile Well Extension       63665           NaN   
4          4     15 Mile Silver City       65760           NaN   

                                     OWNER(S) Equity ownership %  \
0  Sodeistvie Private Enterprise (Owner) 100%                100   
1               U&D Coal Limited (Owner) 100%                100   
2          Dios Exploration Inc. (Owner) 100%                100   
3               Private Interest (Owner) 100%                100   
4       North Lily Mining Company (Owner) 50%                 50   

  DEVELOPMENT STAGE ACTIVITY STATUS   COMMODITY(S) PRIMARY COMMODITY  ...  \
0        Grassroots        Inactive           Coal              Coal  ...   
1        Grassroots        Inactive           Coal              Coal  ...   
2        Grassroots        Inactive  Gold,Diamonds              Gold  ...   
3        Grassroots        Inactive           Gold              Gold  ...   
4        Grassroots        Inactive    Gold,Silver              Gold  ...   

  Unnamed: 36 Unnamed: 37 Unnamed: 38 Unnamed: 39 Unnamed: 40  Unnamed: 41  \
0         NaN         NaN         NaN         NaN         NaN          NaN   
1         NaN         NaN         NaN         NaN         NaN          NaN   
2         NaN         NaN         NaN         NaN         NaN          NaN   
3         NaN         NaN         NaN         NaN         NaN          NaN   
4         NaN         NaN         NaN         NaN         NaN          NaN   

   Unnamed: 42 Unnamed: 43 Unnamed: 44 PRIO GRID ID  
0          NaN         NaN         NaN       199878  
1          NaN         NaN         NaN        96419  
2          NaN         NaN         NaN       205418  
3          NaN         NaN         NaN        88445  
4          NaN         NaN         NaN       221841  

[5 rows x 46 columns]

In [4]:
subset_df=df_SNL[["PRIO GRID ID", "Property ID",
                    "DEVELOPMENT STAGE","PRIMARY COMMODITY"
                    ]]
print(subset_df.shape)
subset_df

(39370, 4)


PRIO GRID ID Property ID DEVELOPMENT STAGE PRIMARY COMMODITY
0            199878       68289        Grassroots              Coal
1             96419       66178        Grassroots              Coal
2            205418       65525        Grassroots              Gold
3             88445       63665        Grassroots              Gold
4            221841       65760        Grassroots              Gold
...             ...         ...               ...               ...
39365        106057     62906.0         Operating              Gold
39366        173402     61944.0         Operating            Nickel
39367        189948     82901.0               NaN            Nickel
39368        169787     86402.0            Closed              Zinc
39369        101940     65049.0         Operating       Ferrochrome

[39370 rows x 4 columns]

### Read SNL data with SNL property ID with development stage and source date

In [5]:
def process(path):
    tmp = pd.read_excel(path)
    print(tmp.shape)
    tmp = tmp.drop([0,1,2,4,5], axis=0)
    tmp.head()
    df = tmp.iloc[1:,:]
    df.columns = tmp.iloc[0]
    return df

In [6]:
work_full_history = process('../input/development stage with year.xls')
print(work_full_history.shape)
work_full_history.head()

(39866, 4)
(39860, 4)


3                  PROP_NAME PROP_ID    DEV_STAGE YR_SOURCE_DATE
6         Chihuahua Refinery   86959    Operating           2020
7                        101   64665    Operating           2011
8                       1068   79522  Exploration           2020
9   10th of Ramadan Refinery   86452    Operating           2021
10   10th of Ramadan Smelter   86451    Operating           2021

### Merge 2 SNL datasets based on SNL ID

In [7]:
work_full_history["PROP_ID"] =work_full_history["PROP_ID"].astype("int32")
subset_df["Property ID"] =subset_df["Property ID"].astype("int32")

/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18806/3781885345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["Property ID"] =subset_df["Property ID"].astype("int32")


In [8]:
subset_merge= subset_df.merge(work_full_history, left_on="Property ID",right_on="PROP_ID", how="inner")
print(subset_merge.shape)
subset_merge.head()

(39350, 8)


PRIO GRID ID  Property ID DEVELOPMENT STAGE PRIMARY COMMODITY  \
0        199878        68289        Grassroots              Coal   
1         96419        66178        Grassroots              Coal   
2        205418        65525        Grassroots              Gold   
3         88445        63665        Grassroots              Gold   
4        221841        65760        Grassroots              Gold   

                PROP_NAME  PROP_ID   DEV_STAGE YR_SOURCE_DATE  
0           11 RAU-Novaya    68289  Grassroots           2014  
1           12 Mile Creek    66178  Grassroots           2014  
2               14 Karats    65525  Grassroots           2013  
3  14 Mile Well Extension    63665  Grassroots           2013  
4     15 Mile Silver City    65760  Grassroots           2013

In [9]:
subset_merge=subset_merge.drop(columns=["PROP_NAME", "PROP_ID", "DEV_STAGE"])
subset_merge

PRIO GRID ID  Property ID DEVELOPMENT STAGE PRIMARY COMMODITY  \
0            199878        68289        Grassroots              Coal   
1             96419        66178        Grassroots              Coal   
2            205418        65525        Grassroots              Gold   
3             88445        63665        Grassroots              Gold   
4            221841        65760        Grassroots              Gold   
...             ...          ...               ...               ...   
39345        106057        62906         Operating              Gold   
39346        173402        61944         Operating            Nickel   
39347        189948        82901               NaN            Nickel   
39348        169787        86402            Closed              Zinc   
39349        101940        65049         Operating       Ferrochrome   

      YR_SOURCE_DATE  
0               2014  
1               2014  
2               2013  
3               2013  
4               2013  
...              ...  
39345           2014  
39346           2021  
39347           2020  
39348           2021  
39349            NaN  

[39350 rows x 5 columns]

### Building the time-series data


Assign a numerical value to each development stage according to the order of development stage. Thus, we can compare the squential order of development stage according to the rules wrote in introduction aprt.

The larger the value is assigned, the stage is at the later step. For example, "grassroot" is the beginning stage, "Closed" is the last stage. 

Here is the explanation of each development stage:
https://upenn.app.box.com/file/1032975091991z


In [10]:
subset_merge["DEVELOPMENT STAGE"] =subset_merge["DEVELOPMENT STAGE"].replace({
            np.nan:0, 
            'Grassroots':1, 'Exploration':2,  'Advanced Exploration':3,
            'Target Outline':4, 'Reserves Development':5, 'Advanced Exploration':6, 'Prefeas/Scoping':7,
            'Feasibility':8,  'Feasibility Started':9,'Feasibility Complete':10, 
             'Preproduction': 11, 'Construction Planned':12, 'Construction Started':13, 
             'Commissioning':14, 'Operating':15, 'Satellite':16, 
            'Expansion':17, 'Limited Production':18, 'Residual Production':19, 'Closed':20
})

We only considerate SNL mining which is still not closed in 1990.

In [11]:
subset_drop_year = subset_merge.loc[~((subset_merge["DEVELOPMENT STAGE"]=="Closed")&(subset_merge["YR_SOURCE_DATE"]<1990))]
print(subset_drop_year.shape)
subset_drop_year.head()

(39350, 5)


PRIO GRID ID  Property ID  DEVELOPMENT STAGE PRIMARY COMMODITY  \
0        199878        68289                  1              Coal   
1         96419        66178                  1              Coal   
2        205418        65525                  1              Gold   
3         88445        63665                  1              Gold   
4        221841        65760                  1              Gold   

  YR_SOURCE_DATE  
0           2014  
1           2014  
2           2013  
3           2013  
4           2013

In [12]:
year_ls = [1970, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


Categorize the SNL mining by PRIO GRID id and its year

If there is only one SNL in that category, the development stage of PRIO GRID cell is the only-one development stage

If there are multiple SNL with multiple development stage in that category, the development stage of PRIO GRID cell is the latest development stage with largest assigned value

In [13]:
ls_prio_id = []
year = []
count = []
development_stage = []

tmp_df = subset_drop_year.groupby(['PRIO GRID ID','YR_SOURCE_DATE'])[["Property ID","DEVELOPMENT STAGE"]]

for name_of_group, contents_of_group in tmp_df:
    ls_prio_id.append(name_of_group[0])
    year.append(name_of_group[1])
    count.append(len(contents_of_group))
    if len(contents_of_group) == 1:
        tmp = contents_of_group["DEVELOPMENT STAGE"].to_list()[0]
    if len(contents_of_group)>1:
        tmp = 0 
        for i in contents_of_group["DEVELOPMENT STAGE"].to_list():
            if i > tmp: # choose the latest development stag
                tmp = i
    development_stage.append(tmp)

subset_assigned_value = pd.DataFrame({"PRIO GRID ID": ls_prio_id, "Year":year, "Mining Property Count":count, "Latest development stage":development_stage })
subset_assigned_value.head()

PRIO GRID ID  Year  Mining Property Count  Latest development stage
0         50619  1999                      1                        20
1         50624  1998                      1                         4
2         52063  1994                      1                        15
3         52776  2022                      1                         6
4         52778  2019                      1                         4

In [14]:
np.sort(subset_assigned_value["Year"].unique())

array([1970, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [15]:
gid =[]
year =[]
def expand_function(x):
    for i in year_ls:
        gid.append(x)
        year.append(i)


In [16]:
spine["gid"].apply(lambda x :expand_function(x))

0        None
1        None
2        None
3        None
4        None
         ... 
64813    None
64814    None
64815    None
64816    None
64817    None
Name: gid, Length: 64818, dtype: object

In [17]:
new_spine = pd.DataFrame({"gid":gid, "year":year})
print(new_spine.shape)
new_spine.head()

(2851992, 2)


gid  year
0  49182  1970
1  49182  1980
2  49182  1981
3  49182  1982
4  49182  1983

In [18]:
spine_dev = new_spine.merge(subset_assigned_value, left_on=["gid","year"], right_on=["PRIO GRID ID","Year"], how="left").fillna(0)
spine_dev

gid  year  PRIO GRID ID  Year  Mining Property Count  \
0         49182  1970           0.0   0.0                    0.0   
1         49182  1980           0.0   0.0                    0.0   
2         49182  1981           0.0   0.0                    0.0   
3         49182  1982           0.0   0.0                    0.0   
4         49182  1983           0.0   0.0                    0.0   
...         ...   ...           ...   ...                    ...   
2851987  249344  2018           0.0   0.0                    0.0   
2851988  249344  2019           0.0   0.0                    0.0   
2851989  249344  2020           0.0   0.0                    0.0   
2851990  249344  2021           0.0   0.0                    0.0   
2851991  249344  2022           0.0   0.0                    0.0   

         Latest development stage  
0                             0.0  
1                             0.0  
2                             0.0  
3                             0.0  
4                             0.0  
...                           ...  
2851987                       0.0  
2851988                       0.0  
2851989                       0.0  
2851990                       0.0  
2851991                       0.0  

[2851992 rows x 6 columns]

In [19]:
dict ={}
def form_dict(id, year, count, development_stage_value):
    dict[(int(id),int(year))] = (count, development_stage_value)

In [20]:
spine_dev[["gid","year","Mining Property Count","Latest development stage"]].apply(lambda x: form_dict(*x), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
2851987    None
2851988    None
2851989    None
2851990    None
2851991    None
Length: 2851992, dtype: object

In [21]:
count_2=[]
lateset_year_2 = []

def find_each_latest_year(gid, year, count, stage):
    if count ==0 or stage == 0:
        i = year_ls.index((year))
        if i==0:
            count_2.append(0)
            lateset_year_2.append(0)
        else:
            for j in list(list(year_ls[:i])[::-1]):
                if dict[(gid, j)][0]!=0 and dict[(gid, j)][1]!=0 :
                    count_2.append(dict[(gid, j)][0])
                    lateset_year_2.append(dict[(gid, j)][1])
                    break
                if j==1970:
                    count_2.append(0)
                    lateset_year_2.append(0)
    else:
        count_2.append(count)
        lateset_year_2.append(stage)


In [22]:
spine_dev[["gid", "year", "Mining Property Count","Latest development stage"]].apply(lambda x: find_each_latest_year(*x), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
2851987    None
2851988    None
2851989    None
2851990    None
2851991    None
Length: 2851992, dtype: object

In [23]:
new_spine_with_count_dev_stage =pd.DataFrame({"gid":gid, "year":year,"count":count_2, "lateset_year_development":lateset_year_2},dtype=int)
new_spine_with_count_dev_stage.head()

gid  year  count  lateset_year_development
0  49182  1970      0                         0
1  49182  1980      0                         0
2  49182  1981      0                         0
3  49182  1982      0                         0
4  49182  1983      0                         0

In [24]:
new_spine_with_count_dev_stage["lateset_year_development"] = new_spine_with_count_dev_stage["lateset_year_development"].replace({
            0: np.nan, 
            1: 'Grassroots', 2:'Exploration',  3: 'Advanced Exploration',
            4: 'Target Outline', 5: 'Reserves Developmen', 6:'Advanced Exploration', 7:'Prefeas/Scoping',
            8: 'Feasibility',  9: 'Feasibility Started', 10:'Feasibility Complete', 
            11: 'Preproduction', 12: 'Construction Planned', 13:'Construction Started', 
            14: 'Commissioning', 15: 'Operating', 16: 'Satellite' , 
            17: 'Expansion', 18: 'Limited Production', 19: 'Residual Production', 20: 'Closed'
})
new_spine_with_count_dev_stage = new_spine_with_count_dev_stage.rename(columns={"gid":"PRIO grid id"})

In [25]:
new_spine_with_count_dev_stage.to_csv("../output/snl_yearly_time_series.csv")

### Verification

In [26]:
subset_assigned_value.loc[subset_assigned_value["PRIO GRID ID"]==59983]

PRIO GRID ID  Year  Mining Property Count  Latest development stage
31         59983  2018                      1                         2
32         59983  2020                      1                         7
33         59983  2022                      3                         4

In [27]:
new_spine_with_count_dev_stage[new_spine_with_count_dev_stage["PRIO grid id"]==59983]

PRIO grid id  year  count lateset_year_development
10428         59983  1970      0                      NaN
10429         59983  1980      0                      NaN
10430         59983  1981      0                      NaN
10431         59983  1982      0                      NaN
10432         59983  1983      0                      NaN
10433         59983  1984      0                      NaN
10434         59983  1985      0                      NaN
10435         59983  1986      0                      NaN
10436         59983  1987      0                      NaN
10437         59983  1988      0                      NaN
10438         59983  1989      0                      NaN
10439         59983  1990      0                      NaN
10440         59983  1991      0                      NaN
10441         59983  1992      0                      NaN
10442         59983  1993      0                      NaN
10443         59983  1994      0                      NaN
10444         59983  1995      0                      NaN
10445         59983  1996      0                      NaN
10446         59983  1997      0                      NaN
10447         59983  1998      0                      NaN
10448         59983  1999      0                      NaN
10449         59983  2000      0                      NaN
10450         59983  2001      0                      NaN
10451         59983  2002      0                      NaN
10452         59983  2003      0                      NaN
10453         59983  2004      0                      NaN
10454         59983  2005      0                      NaN
10455         59983  2006      0                      NaN
10456         59983  2007      0                      NaN
10457         59983  2008      0                      NaN
10458         59983  2009      0                      NaN
10459         59983  2010      0                      NaN
10460         59983  2011      0                      NaN
10461         59983  2012      0                      NaN
10462         59983  2013      0                      NaN
10463         59983  2014      0                      NaN
10464         59983  2015      0                      NaN
10465         59983  2016      0                      NaN
10466         59983  2017      0                      NaN
10467         59983  2018      1              Exploration
10468         59983  2019      1              Exploration
10469         59983  2020      1          Prefeas/Scoping
10470         59983  2021      1          Prefeas/Scoping
10471         59983  2022      3           Target Outline